# coasts and boundaries

In [1]:
import numpy as np
import pandas as pd

In [2]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, save
output_notebook()
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout

Loading BokehJS ...

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [4]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

On the basis of both recensions of the Geography, the localities of the Iberian peninsula are plotted and compared.

## import resources (omega)

In [5]:
OmegaJ = pd.read_json('../data/OmegaStructure2.json')
XiJ = pd.read_json('../data/XiStructure2.json')

## transform .json to df

In [6]:
%run geographytools.py

In [7]:
df=OmegaJ["chapters"][0]
om=json_normalize(df,"section")

In [8]:
def findTS(s):
    out=""
    for i,r in om.iterrows():
        sci=r["sec_ID"]
        if sci in s:
            out=r["type_sec"]
            
            exit
    return(out)
findTS("2.04.01.01")

'area presentation'

In [9]:
def Js2Geodf(df): 
# df=OmegaJ["chapters"][0]
    om=json_normalize(df,"section")
    om=om.dropna(subset=["sec_part"])
    l=[]
    for i,x in om.iterrows():
        k={"type_sec":x["type_sec"]}
        l.append([x["sec_part"]])
#
    listItems=list(flatten_list(l))
    dfout=pd.DataFrame(listItems)
    dfout["type_sec"]=dfout.apply(lambda x: findTS(x["ID"]),axis=1)
    return(dfout)

In [10]:
dfOmega=Js2Geodf(OmegaJ["chapters"][0])

## Translate Greek to decimal coordinates

In [11]:
gfrac={"":0,"Lς":2/3,"ςL":2/3,"ιβ":1/12,"ς":1/6,"δ":1/4,"γ":1/3,"γιβ":5/12,"L":1/2,"Lιβ":7/12,"γο":2/3,"Lδ":3/4,"Lγ":5/6,"Lγιβ":11/12,"η":1/8}
gint={"":0,"α":1,"β":2,"γ":3,"δ":4,"ε":5,"ς":6,"ζ":7,"η":8,"θ":9,"ι":10,"κ":20,"λ":30,"μ":40}

In [12]:
def reformatCoord(row,longLat, xy='coord_x'):
    if type(row[xy]) == dict:
        return (row[xy][longLat]['integer'].strip(),row[xy][longLat]['fraction'].strip())
    else:
        return False
    
def translate(row):
    if type(row)==tuple:
        try:
            temp_frac = gfrac[row[1]]
        except:
            temp_frac = gint[row[1]]
        try:
            if len(row[0]) == 1:
                temp_int = gint[row[0]]
            elif len(row[0]) == 2:
                temp_int = gint[row[0][0]] + gint[row[0][1]]
            elif row[0] == '':
                temp_int = 0
        except:
            temp_int = None
        return temp_int + temp_frac

Omega recension

In [13]:
dfTemp = dfOmega.copy()
dfTemp['longitude'] = dfOmega.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(translate)
dfTemp['latitude'] = dfOmega.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(translate)

In [14]:
dfTemp;

## select coastal localities

I select only type_sec: "coast section"

In [15]:
dfOmegaCoasts = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality')]

In "coast sections", Ptolemy also give coordinates for the sources of river for instance, but these points are not part of the coast. So, we have to remove from every coast section places that are not actually on the coast, i.e. the following categories: river source and river path.

In [16]:
dfOmegaCoasts = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality')]
dfOmegaCoasts = dfOmegaCoasts[dfOmegaCoasts.category.apply(lambda row: row not in ['river path','river source'])]

In [17]:
dfOmegaCoasts.head()

,ID,category,coord,people,text,toponym,type,type_sec,longitude,latitude
4,2.04.03.01,"[river mouth, boundary]","{'long': {'integer': 'δ', 'fraction': 'ιβ'}, 'lat': {'integer': 'λζ', 'fraction': 'γο'}}",Turdetani,NaN,<τὸ δυσμικώτερον στόμα Ἄνα τοῦ ποταμοῦ>,locality,coast section,4.083333,37.666667
5,2.04.03.02,river mouth,"{'long': {'integer': 'δ', 'fraction': 'γ'}, 'lat': {'integer': 'λζ', 'fraction': 'L'}}",Turdetani,NaN,τὸ ἀνατολικώτερον στόμα Ἄνα τοῦ ποταμοῦ,locality,coast section,4.333333,37.500000
7,2.04.03.04,"[river path, boundary]","{'long': {'integer': 'θ', 'fraction': ''}, 'lat': {'integer': 'λθ', 'fraction': ''}}",,NaN,τὸ πρὸς τῷ πέρατι τῆς Λουσιτανίας τοῦ ποταμοῦ μέρος,locality,coast section,9.000000,39.000000
8,2.04.03.07,boundary,"{'long': {'integer': 'ιβ', 'fraction': ''}, 'lat': {'integer': 'λζ', 'fraction': 'δ'}}",,NaN,καὶ ἡ ἐντεῦθεν γραμμὴ γραφομένη παρὰ τὴν Ταρρακωνησίαν ὡς ἐπὶ τὸ Βαλιαρικὸν πέλαγος πέρας ἔχουσα,locality,coast section,12.000000,37.250000
15,2.04.05.04,city,"{'long': {'integer': 'δ', 'fraction': 'γο'}, 'lat': {'integer': 'λζ', 'fraction': 'γ'}}",Turdetani,NaN,Ὄνοβα Λιστουρία,locality,coast section,4.666667,37.333333


First problem: places with multiple categories ("river mouth, boundary") are not taken into account when filtering and I don't know how to solve the problem.

## map omega

In [18]:
p = figure(title='Omega Recension', width=800, height=600, x_range=(1.5, 22), y_range=(35.5, 47), tools= [BoxZoomTool(), WheelZoomTool(), ResetTool()])

# coasts 
p.line(dfOmegaCoasts['longitude'],dfOmegaCoasts['latitude'], color='grey')

# localities
p.circle(np.array(dfTemp['longitude']),np.array(dfTemp['latitude']), fill_color='red',size=4, fill_alpha=.9, line_color='red',line_alpha=0.7)
p.circle(np.array(dfOmegaCoasts['longitude']),np.array(dfOmegaCoasts['latitude']), fill_color='black',size=5, fill_alpha=.9, line_color='black',line_alpha=0.7)

show(p);

First series of problems related to the filtering:

- inland localities with two categories and in type_sec "coastal section" are still in the dataframe;
- the city of Baria (ΙD = 2.04.08.08), located on the coast, is not integrated to a "coastal section" by Ptolemy, so does not appear as "coastal" here;
- problem for filtering the boundary points: they have two categories ("river mouth" & "boundary" for instance) except one, on the Mediterranean coast between Baetica and Tarraconensis, which is only "boundary". I don't know how to deal with this automatically for the filtering.


### problem with the ordering of the sections

In [19]:
# parts of coasts and inland boundaries
a = dfOmegaCoasts[:29] #blau
b = dfOmegaCoasts[29:70] #rot
c = dfOmegaCoasts[70:99] #grün
pyrenees = dfTemp[(dfTemp.ID == '2.06.11.03')] #orange
baria = dfTemp[(dfTemp.ID == '2.04.08.08')] #grey

p = figure(title='Omega Recension', width=800, height=600, x_range=(1.5, 22), y_range=(35.5, 47), tools= [BoxZoomTool(), WheelZoomTool(), ResetTool()])

# coasts 
p.line(a['longitude'],a['latitude'], color='grey')
p.line(b['longitude'],b['latitude'], color='grey')
p.line(c['longitude'],c['latitude'], color='grey')

# localities
p.circle(np.array(a['longitude']),np.array(a['latitude']), fill_color='blue',size=5, fill_alpha=.9, line_color='blue',line_alpha=0.7)
p.circle(np.array(b['longitude']),np.array(b['latitude']), fill_color='red',size=5, fill_alpha=.9, line_color='red',line_alpha=0.7)
p.circle(np.array(c['longitude']),np.array(c['latitude']), fill_color='green',size=5, fill_alpha=.9, line_color='green',line_alpha=0.7)
p.circle(np.array(pyrenees['longitude']),np.array(pyrenees['latitude']), fill_color='orange',size=5, fill_alpha=.9, line_color='orange',line_alpha=0.7)
p.circle(np.array(baria['longitude']),np.array(baria['latitude']), fill_color='grey',size=5, fill_alpha=.9, line_color='grey',line_alpha=0.7)

#additional point in the Pyrenees
p.circle(19,43.167, fill_color='black',size=5, fill_alpha=.9, line_color='black',line_alpha=0.7)

# additional line for the Pyrenees
p.line([15,17,19,20.333], [45.833,43,43.167,42.333], line_alpha=0.8, color="black")


show(p);

The blue coast is described in chap. 2.4 of the catalogue, the red in chap. 2.5 and 2.6, and the green in chap. 2.6.

The order in the catalogue is:
- beginning of the blue coast from west
- Baria (last city on the blue coast, near the eastern end of the blue coastline)
- rest of the blue coast from west to east (except Baria)
- red coast from south to north
- point in the middle of the Pyrenees (orange here on the map, ID = 2.06.11.03)
- green coast from south to north

Finally, in chap. 2.7, which is not part of the Iberian chapters, an additional point in the Pyrenees: long. 19°, lat. 43°10', here in black on the map.